# **Visit Report Summary**

### **Doctor Visit Summary**

In [1]:
import pyodbc
import pandas as pd
import psycopg2
import time
from datetime import datetime
import calendar
import openpyxl

In [2]:
postg_conn = psycopg2.connect(
    host="aws-0-ap-south-1.pooler.supabase.com",
    database="postgres",
    user="powerbi.hnwlvauqeoolcvyuotrh",    
    password="OlVui6cY8p6K", 
    port="5432"
)

In [3]:
ddf = pd.read_sql("""SELECT CAST(x.created_at AS date) AS Date,
                     x.salesperson_fkey,
                     y.full_name as SalesPerson,
                     a.docid,
                     a.doctor_name,
                     CASE WHEN a.doctor_name ILIKE '%NEW CUST%' THEN 'NEW CUST' ELSE NULL END AS New_Cust,
                     z.street as Address,
                     address_fkey,
                     z.city,
                     z.zipcode,
                     x.longitude,
                     x.latitude,
                     z.longitude AS Actual_longitude,
                     z.latitude AS Actual_latitude,
                     x.notes,
                     x.created_at as created_at,
                     ROUND(y.manager_fkey)::INT as manager_fkey
                     FROM doctor_address_visits x
                     LEFT JOIN users y ON x.salesperson_fkey = y.id
                     LEFT JOIN doctor_addresses z ON x.address_fkey = z.id
                     LEFT JOIN doctors a ON z.doctor_fkey = a.id""", postg_conn)

ddf.head()

C:\Users\suradmin\AppData\Local\Temp\ipykernel_13728\400344620.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ddf = pd.read_sql("""SELECT CAST(x.created_at AS date) AS Date,


,date,salesperson_fkey,salesperson,docid,doctor_name,new_cust,address,address_fkey,city,zipcode,longitude,latitude,actual_longitude,actual_latitude,notes,created_at,manager_fkey
0,2025-01-16,237,ARINDAM SAHA,CEN-01,HO,None,"DG block,New town,HO",2,Kolkata,700156,88.476172,22.579553,None,None,HO staff,2025-01-16 12:22:31.606730+00:00,200.0
1,2025-01-16,222,DIPANKAR ROY KARMAKAR,DOC-00924,SUBHASISH BISWAS,None,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",4,Kolkata,700092,88.363652,22.486545,None,None,,2025-01-16 16:07:33.324867+00:00,207.0
2,2025-01-16,222,DIPANKAR ROY KARMAKAR,DOC-00924,SUBHASISH BISWAS,None,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",4,Kolkata,700092,88.363638,22.486586,None,None,,2025-01-16 16:11:26.025069+00:00,207.0
3,2025-01-17,223,SAYANTAN BOSE,DOC-29443,SHASHANKA DHANUKA,None,Ballygunge,5,Kolkata,700016,88.375949,22.526766,None,None,,2025-01-17 04:14:48.712002+00:00,207.0
4,2025-01-17,223,SAYANTAN BOSE,DOC-13862,D S CHAKRABORTY,None,36 ramkrishna lane,11,Kolkata,700078,88.378202,22.504825,None,None,,2025-01-17 05:21:31.511517+00:00,207.0


In [4]:
def location_status(row):
    if row['longitude'] == row['actual_longitude'] and row['latitude'] == row['actual_latitude']:
        return True
    else:
        return False

ddf['location_status'] = ddf.apply(location_status, axis= 1)
ddf.head()

,date,salesperson_fkey,salesperson,docid,doctor_name,new_cust,address,address_fkey,city,zipcode,longitude,latitude,actual_longitude,actual_latitude,notes,created_at,manager_fkey,location_status
0,2025-01-16,237,ARINDAM SAHA,CEN-01,HO,None,"DG block,New town,HO",2,Kolkata,700156,88.476172,22.579553,None,None,HO staff,2025-01-16 12:22:31.606730+00:00,200.0,False
1,2025-01-16,222,DIPANKAR ROY KARMAKAR,DOC-00924,SUBHASISH BISWAS,None,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",4,Kolkata,700092,88.363652,22.486545,None,None,,2025-01-16 16:07:33.324867+00:00,207.0,False
2,2025-01-16,222,DIPANKAR ROY KARMAKAR,DOC-00924,SUBHASISH BISWAS,None,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",4,Kolkata,700092,88.363638,22.486586,None,None,,2025-01-16 16:11:26.025069+00:00,207.0,False
3,2025-01-17,223,SAYANTAN BOSE,DOC-29443,SHASHANKA DHANUKA,None,Ballygunge,5,Kolkata,700016,88.375949,22.526766,None,None,,2025-01-17 04:14:48.712002+00:00,207.0,False
4,2025-01-17,223,SAYANTAN BOSE,DOC-13862,D S CHAKRABORTY,None,36 ramkrishna lane,11,Kolkata,700078,88.378202,22.504825,None,None,,2025-01-17 05:21:31.511517+00:00,207.0,False


In [5]:
ddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26082 entries, 0 to 26081
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   date              26082 non-null  object             
 1   salesperson_fkey  26082 non-null  int64              
 2   salesperson       26082 non-null  object             
 3   docid             26082 non-null  object             
 4   doctor_name       26082 non-null  object             
 5   new_cust          6515 non-null   object             
 6   address           26082 non-null  object             
 7   address_fkey      26082 non-null  int64              
 8   city              26082 non-null  object             
 9   zipcode           26082 non-null  object             
 10  longitude         26082 non-null  float64            
 11  latitude          26082 non-null  float64            
 12  actual_longitude  0 non-null      object             
 13  a

In [6]:
ddf['date'] = pd.to_datetime(ddf['date'])
ddf['manager_fkey'] = ddf['manager_fkey'].astype('Int64')
ddf['actual_longitude'] = ddf['actual_longitude'].astype(float)
ddf['actual_latitude'] = ddf['actual_latitude'].astype(float)

In [7]:
ddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26082 entries, 0 to 26081
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   date              26082 non-null  datetime64[ns]     
 1   salesperson_fkey  26082 non-null  int64              
 2   salesperson       26082 non-null  object             
 3   docid             26082 non-null  object             
 4   doctor_name       26082 non-null  object             
 5   new_cust          6515 non-null   object             
 6   address           26082 non-null  object             
 7   address_fkey      26082 non-null  int64              
 8   city              26082 non-null  object             
 9   zipcode           26082 non-null  object             
 10  longitude         26082 non-null  float64            
 11  latitude          26082 non-null  float64            
 12  actual_longitude  0 non-null      float64            
 13  a

In [8]:
from datetime import timedelta

ddf['realtime'] = ddf['created_at'] + timedelta(hours= 5, minutes= 30)
ddf['realtime'] = ddf['realtime'].dt.time
ddf.head()

,date,salesperson_fkey,salesperson,docid,doctor_name,new_cust,address,address_fkey,city,zipcode,longitude,latitude,actual_longitude,actual_latitude,notes,created_at,manager_fkey,location_status,realtime
0,2025-01-16,237,ARINDAM SAHA,CEN-01,HO,None,"DG block,New town,HO",2,Kolkata,700156,88.476172,22.579553,NaN,NaN,HO staff,2025-01-16 12:22:31.606730+00:00,200,False,17:52:31.606730
1,2025-01-16,222,DIPANKAR ROY KARMAKAR,DOC-00924,SUBHASISH BISWAS,None,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",4,Kolkata,700092,88.363652,22.486545,NaN,NaN,,2025-01-16 16:07:33.324867+00:00,207,False,21:37:33.324867
2,2025-01-16,222,DIPANKAR ROY KARMAKAR,DOC-00924,SUBHASISH BISWAS,None,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",4,Kolkata,700092,88.363638,22.486586,NaN,NaN,,2025-01-16 16:11:26.025069+00:00,207,False,21:41:26.025069
3,2025-01-17,223,SAYANTAN BOSE,DOC-29443,SHASHANKA DHANUKA,None,Ballygunge,5,Kolkata,700016,88.375949,22.526766,NaN,NaN,,2025-01-17 04:14:48.712002+00:00,207,False,09:44:48.712002
4,2025-01-17,223,SAYANTAN BOSE,DOC-13862,D S CHAKRABORTY,None,36 ramkrishna lane,11,Kolkata,700078,88.378202,22.504825,NaN,NaN,,2025-01-17 05:21:31.511517+00:00,207,False,10:51:31.511517


In [9]:
import math

def haversine_distance(lat1, lon1, lat2, lon2):
    # Check for None or missing values
    if None in (lat1, lon1, lat2, lon2):
        return None
    
    # Radius of the Earth in meters
    R = 6371000  
    
    # Convert degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Differences
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad
    
    # Haversine formula
    a = (math.sin(delta_lat / 2) ** 2 +
         math.cos(lat1_rad) * math.cos(lat2_rad) *
         math.sin(delta_lon / 2) ** 2)
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Distance in meters
    distance = R * c
    return distance


ddf['distance_meters'] = ddf.apply(lambda row: haversine_distance(row['latitude'],
                                                                  row['longitude'],
                                                                  row['actual_latitude'],
                                                                  row['actual_longitude']
                                    ), axis=1)
ddf.head()

,date,salesperson_fkey,salesperson,docid,doctor_name,new_cust,address,address_fkey,city,zipcode,longitude,latitude,actual_longitude,actual_latitude,notes,created_at,manager_fkey,location_status,realtime,distance_meters
0,2025-01-16,237,ARINDAM SAHA,CEN-01,HO,None,"DG block,New town,HO",2,Kolkata,700156,88.476172,22.579553,NaN,NaN,HO staff,2025-01-16 12:22:31.606730+00:00,200,False,17:52:31.606730,NaN
1,2025-01-16,222,DIPANKAR ROY KARMAKAR,DOC-00924,SUBHASISH BISWAS,None,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",4,Kolkata,700092,88.363652,22.486545,NaN,NaN,,2025-01-16 16:07:33.324867+00:00,207,False,21:37:33.324867,NaN
2,2025-01-16,222,DIPANKAR ROY KARMAKAR,DOC-00924,SUBHASISH BISWAS,None,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",4,Kolkata,700092,88.363638,22.486586,NaN,NaN,,2025-01-16 16:11:26.025069+00:00,207,False,21:41:26.025069,NaN
3,2025-01-17,223,SAYANTAN BOSE,DOC-29443,SHASHANKA DHANUKA,None,Ballygunge,5,Kolkata,700016,88.375949,22.526766,NaN,NaN,,2025-01-17 04:14:48.712002+00:00,207,False,09:44:48.712002,NaN
4,2025-01-17,223,SAYANTAN BOSE,DOC-13862,D S CHAKRABORTY,None,36 ramkrishna lane,11,Kolkata,700078,88.378202,22.504825,NaN,NaN,,2025-01-17 05:21:31.511517+00:00,207,False,10:51:31.511517,NaN


In [10]:
def custtype(row):
    if "DOC" in str(row['docid']):
        return "Doctor"
    elif pd.notna(row['new_cust']):
        return "New Cust"
    else:
        return "DC"

ddf['cust_type'] = ddf.apply(custtype, axis=1)
ddf.head()

,date,salesperson_fkey,salesperson,docid,doctor_name,new_cust,address,address_fkey,city,zipcode,...,latitude,actual_longitude,actual_latitude,notes,created_at,manager_fkey,location_status,realtime,distance_meters,cust_type
0,2025-01-16,237,ARINDAM SAHA,CEN-01,HO,None,"DG block,New town,HO",2,Kolkata,700156,...,22.579553,NaN,NaN,HO staff,2025-01-16 12:22:31.606730+00:00,200,False,17:52:31.606730,NaN,DC
1,2025-01-16,222,DIPANKAR ROY KARMAKAR,DOC-00924,SUBHASISH BISWAS,None,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",4,Kolkata,700092,...,22.486545,NaN,NaN,,2025-01-16 16:07:33.324867+00:00,207,False,21:37:33.324867,NaN,Doctor
2,2025-01-16,222,DIPANKAR ROY KARMAKAR,DOC-00924,SUBHASISH BISWAS,None,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",4,Kolkata,700092,...,22.486586,NaN,NaN,,2025-01-16 16:11:26.025069+00:00,207,False,21:41:26.025069,NaN,Doctor
3,2025-01-17,223,SAYANTAN BOSE,DOC-29443,SHASHANKA DHANUKA,None,Ballygunge,5,Kolkata,700016,...,22.526766,NaN,NaN,,2025-01-17 04:14:48.712002+00:00,207,False,09:44:48.712002,NaN,Doctor
4,2025-01-17,223,SAYANTAN BOSE,DOC-13862,D S CHAKRABORTY,None,36 ramkrishna lane,11,Kolkata,700078,...,22.504825,NaN,NaN,,2025-01-17 05:21:31.511517+00:00,207,False,10:51:31.511517,NaN,Doctor


In [11]:
udf = pd.read_sql("""SELECT id, manager_fkey, full_name FROM users""", postg_conn)
tdf = pd.merge(ddf, udf, left_on= 'manager_fkey', right_on= 'id', how= 'left')
xdf = tdf[['date', 'salesperson_fkey', 'created_at', 'salesperson', 'full_name', 'cust_type', 'docid', 'doctor_name', 'address_fkey', 'address', 'city', 'zipcode', 'longitude',
           'latitude', 'distance_meters', 'location_status', 'notes', 'realtime']].rename(columns= {'full_name' : 'ReportingManager'})
xdf.head()

C:\Users\suradmin\AppData\Local\Temp\ipykernel_13728\894271729.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  udf = pd.read_sql("""SELECT id, manager_fkey, full_name FROM users""", postg_conn)


,date,salesperson_fkey,created_at,salesperson,ReportingManager,cust_type,docid,doctor_name,address_fkey,address,city,zipcode,longitude,latitude,distance_meters,location_status,notes,realtime
0,2025-01-16,237,2025-01-16 12:22:31.606730+00:00,ARINDAM SAHA,NIREN KAUL,DC,CEN-01,HO,2,"DG block,New town,HO",Kolkata,700156,88.476172,22.579553,NaN,False,HO staff,17:52:31.606730
1,2025-01-16,222,2025-01-16 16:07:33.324867+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,DOC-00924,SUBHASISH BISWAS,4,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",Kolkata,700092,88.363652,22.486545,NaN,False,,21:37:33.324867
2,2025-01-16,222,2025-01-16 16:11:26.025069+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,DOC-00924,SUBHASISH BISWAS,4,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",Kolkata,700092,88.363638,22.486586,NaN,False,,21:41:26.025069
3,2025-01-17,223,2025-01-17 04:14:48.712002+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,DOC-29443,SHASHANKA DHANUKA,5,Ballygunge,Kolkata,700016,88.375949,22.526766,NaN,False,,09:44:48.712002
4,2025-01-17,223,2025-01-17 05:21:31.511517+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,DOC-13862,D S CHAKRABORTY,11,36 ramkrishna lane,Kolkata,700078,88.378202,22.504825,NaN,False,,10:51:31.511517


In [12]:
xdf.to_csv(r"C:\Users\suradmin\OneDrive - Suraksha Diagnostic Limited\Desktop\rnd.csv", index= False)

### **Customer wise Visit**

In [13]:
# cdf = tdf[['date', 'created_at', 'salesperson', 'full_name', 'cust_type', 'docid', 'doctor_name', 'address_fkey', 'address', 'city', 'zipcode', 'longitude',
#            'latitude', 'distance_meters', 'location_status', 'notes', 'realtime']]

# # adf = pd.read_sql("""SELECT id, doctor_fkey FROM doctor_addresses""", postg_conn)

# # cdf = pd.merge(cdf, adf, left_on= 'address_fkey', right_on= 'id', how= 'left')
# # cdf = cdf[['date', 'salesperson', 'full_name', 'cust_type', 'doctor_fkey', 'docid', 'doctor_name', 'address', 'city', 'zipcode', 'longitude',
# #            'latitude', 'distance_meters', 'location_status', 'notes', 'realtime']].rename(columns= {'full_name' : 'ReportingManager'})
# cdf.columns()
adf = pd.read_sql("""SELECT id, doctor_fkey FROM doctor_addresses""", postg_conn)
ydf = pd.merge(xdf, adf, left_on= 'address_fkey', right_on= 'id', how= 'left')
cdf = ydf[['date', 'salesperson_fkey', 'salesperson', 'ReportingManager', 'cust_type', 'doctor_fkey', 'docid', 'doctor_name', 'address', 'city', 'zipcode', 'longitude',
           'latitude', 'distance_meters', 'location_status', 'notes', 'realtime']]

cdf.head()

C:\Users\suradmin\AppData\Local\Temp\ipykernel_13728\1047465252.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  adf = pd.read_sql("""SELECT id, doctor_fkey FROM doctor_addresses""", postg_conn)


,date,salesperson_fkey,salesperson,ReportingManager,cust_type,doctor_fkey,docid,doctor_name,address,city,zipcode,longitude,latitude,distance_meters,location_status,notes,realtime
0,2025-01-16,237,ARINDAM SAHA,NIREN KAUL,DC,2865,CEN-01,HO,"DG block,New town,HO",Kolkata,700156,88.476172,22.579553,NaN,False,HO staff,17:52:31.606730
1,2025-01-16,222,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,SUBHASISH BISWAS,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",Kolkata,700092,88.363652,22.486545,NaN,False,,21:37:33.324867
2,2025-01-16,222,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,SUBHASISH BISWAS,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",Kolkata,700092,88.363638,22.486586,NaN,False,,21:41:26.025069
3,2025-01-17,223,SAYANTAN BOSE,SUMAN PAUL,Doctor,2251,DOC-29443,SHASHANKA DHANUKA,Ballygunge,Kolkata,700016,88.375949,22.526766,NaN,False,,09:44:48.712002
4,2025-01-17,223,SAYANTAN BOSE,SUMAN PAUL,Doctor,2049,DOC-13862,D S CHAKRABORTY,36 ramkrishna lane,Kolkata,700078,88.378202,22.504825,NaN,False,,10:51:31.511517


In [14]:
zaz = cdf[['date', 'salesperson', 'ReportingManager', 'cust_type', 'doctor_fkey']]
cvf = zaz.groupby(['date', 'salesperson', 'ReportingManager', 'cust_type'])['doctor_fkey'].count()
cvf = pd.DataFrame(cvf).rename(columns= {'doctor_fkey' : 'Total_Visit'})
cvf

Total_Visit
date       salesperson           ReportingManager cust_type             
2025-01-16 ARINDAM SAHA          NIREN KAUL       DC                   1
           DIPANKAR ROY KARMAKAR SUMAN PAUL       Doctor               2
2025-01-17 AVIJIT SINGH          VIPIN KUMAR      Doctor              15
           DEBAJIT PAL CHOUDHURY SHANTANU DEB     Doctor               3
           DIPANKAR ROY KARMAKAR SUMAN PAUL       Doctor               5
...                                                                  ...
2025-07-11 SUDIP CHAKRABORTY     SAUGATA GHOSH    New Cust             6
           SWARUP SAHA           PANKAJ CHOUDHURY Doctor               8
           TANMAY SARKAR         DEBASIS DE       Doctor               5
                                                  New Cust             1
2025-07-12 SOUVIK DEY            SHANTANU DEB     Doctor               1

[6057 rows x 1 columns]

In [15]:
cvf.to_csv(r"C:\Users\suradmin\OneDrive - Suraksha Diagnostic Limited\Desktop\rnd2.csv")

### **Count of Visit made each Month**

In [16]:
edf = zaz
edf['FM'] = pd.to_datetime(edf['date']).dt.strftime("%b'%y")
edf.head()

C:\Users\suradmin\AppData\Local\Temp\ipykernel_13728\3539789085.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edf['FM'] = pd.to_datetime(edf['date']).dt.strftime("%b'%y")


,date,salesperson,ReportingManager,cust_type,doctor_fkey,FM
0,2025-01-16,ARINDAM SAHA,NIREN KAUL,DC,2865,Jan'25
1,2025-01-16,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,Jan'25
2,2025-01-16,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,Jan'25
3,2025-01-17,SAYANTAN BOSE,SUMAN PAUL,Doctor,2251,Jan'25
4,2025-01-17,SAYANTAN BOSE,SUMAN PAUL,Doctor,2049,Jan'25


In [17]:
emf = edf.groupby(['FM', 'salesperson', 'ReportingManager', 'cust_type'])['doctor_fkey'].count()
emf = pd.DataFrame(emf).rename(columns= {'doctor_fkey' : 'Total_Visit'})
emf

Total_Visit
FM     salesperson     ReportingManager cust_type             
Apr'25 ANANYA MAJUMDER SHANTANU DEB     DC                   1
                                        Doctor              37
                                        New Cust            92
       AVIJIT SINGH    VIPIN KUMAR      DC                  29
                                        Doctor             190
...                                                        ...
May'25 SWARUP SAHA     PANKAJ CHOUDHURY Doctor             166
                                        New Cust            21
       TANMAY SARKAR   DEBASIS DE       DC                  12
                                        Doctor             153
                                        New Cust            27

[485 rows x 1 columns]

In [18]:
emf.to_csv(r"C:\Users\suradmin\OneDrive - Suraksha Diagnostic Limited\Desktop\rnd3.csv")

# **Time wise Visit Report**

### **Count of Visit made During 3 Time slot**

In [19]:
def time_bucket(dt):
    hour = dt.hour
    if 9 <= hour < 13:
        return "9:30AM-1:00PM"
    elif 13 <= hour < 17:
        return "1:00PM-5:00PM"
    elif 17 <= hour < 21:
        return "5:00PM-9:00PM"
    else:
        return "Beyond"


zdf = ydf[['date', 'salesperson_fkey', 'created_at', 'salesperson', 'ReportingManager', 'cust_type', 'doctor_fkey', 'docid', 'doctor_name', 'address', 'city', 'zipcode', 'longitude',
           'latitude', 'distance_meters', 'location_status', 'notes', 'realtime']]

zdf['time_bucket'] = zdf['created_at'].apply(time_bucket)
zdf.head()

C:\Users\suradmin\AppData\Local\Temp\ipykernel_13728\2686160798.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zdf['time_bucket'] = zdf['created_at'].apply(time_bucket)


,date,salesperson_fkey,created_at,salesperson,ReportingManager,cust_type,doctor_fkey,docid,doctor_name,address,city,zipcode,longitude,latitude,distance_meters,location_status,notes,realtime,time_bucket
0,2025-01-16,237,2025-01-16 12:22:31.606730+00:00,ARINDAM SAHA,NIREN KAUL,DC,2865,CEN-01,HO,"DG block,New town,HO",Kolkata,700156,88.476172,22.579553,NaN,False,HO staff,17:52:31.606730,9:30AM-1:00PM
1,2025-01-16,222,2025-01-16 16:07:33.324867+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,SUBHASISH BISWAS,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",Kolkata,700092,88.363652,22.486545,NaN,False,,21:37:33.324867,1:00PM-5:00PM
2,2025-01-16,222,2025-01-16 16:11:26.025069+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,SUBHASISH BISWAS,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",Kolkata,700092,88.363638,22.486586,NaN,False,,21:41:26.025069,1:00PM-5:00PM
3,2025-01-17,223,2025-01-17 04:14:48.712002+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2251,DOC-29443,SHASHANKA DHANUKA,Ballygunge,Kolkata,700016,88.375949,22.526766,NaN,False,,09:44:48.712002,Beyond
4,2025-01-17,223,2025-01-17 05:21:31.511517+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2049,DOC-13862,D S CHAKRABORTY,36 ramkrishna lane,Kolkata,700078,88.378202,22.504825,NaN,False,,10:51:31.511517,Beyond


In [20]:
ttf = zdf[['date', 'salesperson', 'ReportingManager', 'cust_type', 'time_bucket', 'doctor_fkey']]
ttf.head()

,date,salesperson,ReportingManager,cust_type,time_bucket,doctor_fkey
0,2025-01-16,ARINDAM SAHA,NIREN KAUL,DC,9:30AM-1:00PM,2865
1,2025-01-16,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1:00PM-5:00PM,1143
2,2025-01-16,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1:00PM-5:00PM,1143
3,2025-01-17,SAYANTAN BOSE,SUMAN PAUL,Doctor,Beyond,2251
4,2025-01-17,SAYANTAN BOSE,SUMAN PAUL,Doctor,Beyond,2049


In [21]:
twc = ttf.groupby(['date', 'salesperson', 'ReportingManager', 'cust_type', 'time_bucket'])['doctor_fkey'].count()
twc = pd.DataFrame(twc).rename(columns= {'doctor_fkey' : 'Total_Visit'})
twc

Total_Visit
date       salesperson           ReportingManager cust_type time_bucket               
2025-01-16 ARINDAM SAHA          NIREN KAUL       DC        9:30AM-1:00PM            1
           DIPANKAR ROY KARMAKAR SUMAN PAUL       Doctor    1:00PM-5:00PM            2
2025-01-17 AVIJIT SINGH          VIPIN KUMAR      Doctor    1:00PM-5:00PM            8
                                                            9:30AM-1:00PM            2
                                                            Beyond                   5
...                                                                                ...
2025-07-11 SWARUP SAHA           PANKAJ CHOUDHURY Doctor    Beyond                   3
           TANMAY SARKAR         DEBASIS DE       Doctor    9:30AM-1:00PM            2
                                                            Beyond                   3
                                                  New Cust  Beyond                   1
2025-07-12 SOUVIK DEY            SHANTANU DEB     Doctor    Beyond                   1

[10594 rows x 1 columns]

In [22]:
twc.to_csv(r"C:\Users\suradmin\OneDrive - Suraksha Diagnostic Limited\Desktop\rnd4.csv")

### **Doctor wise Visit Count**

In [23]:
vcf = zdf[['date', 'docid', 'doctor_name', 'salesperson', 'ReportingManager', 'doctor_fkey']]
dvf = vcf.groupby(['date', 'docid', 'doctor_name', 'salesperson', 'ReportingManager'])['doctor_fkey'].count()
dvf = pd.DataFrame(dvf).rename(columns= {'doctor_fkey' : 'TotalVisit'})
dvf

TotalVisit
date       docid     doctor_name      salesperson           ReportingManager               
2025-01-16 CEN-01    HO               ARINDAM SAHA          NIREN KAUL                    1
           DOC-00924 SUBHASISH BISWAS DIPANKAR ROY KARMAKAR SUMAN PAUL                    2
2025-01-17 DOC-00034 K. D. BISWAS     SAMIR PAL             SUVENDU CHAKRABORTY           4
           DOC-00058 L K BAID         RABI SHANKAR SAHA     ADRISH DAS                    1
           DOC-00068 A N MUKHERJEE    RABI SHANKAR SAHA     ADRISH DAS                    2
...                                                                                     ...
2025-07-11 DOC-85016 MANISH AGARWAL   PALASH THAKURIA       PANKAJ CHOUDHURY              3
           KAS-0001  KAS NEW CUST     PULAK MAITY           SUMAN PAUL                    1
                                      SAYANTAN BOSE         SUMAN PAUL                    1
           KES-0001  KES NEW CUST     RABI SHANKAR SAHA     ADRISH DAS                    1
2025-07-12 DOC-25750 NABAJIBAN MANDAL SOUVIK DEY            SHANTANU DEB                  1

[19577 rows x 1 columns]

In [24]:
dvf.to_csv(r"C:\Users\suradmin\OneDrive - Suraksha Diagnostic Limited\Desktop\rnd5.csv")

### **Days wise Doctor visit remark**

In [25]:
# rdf = pd.read_sql("""SELECT CAST(x.created_at AS date) AS Date,
#                      y.full_name as SalesPerson,
#                      a.docid,
#                      a.doctor_name,
#                      CASE WHEN a.doctor_name ILIKE '%NEW CUST%' THEN 'NEW CUST' ELSE NULL END AS New_Cust,
#                      x.notes
#                      FROM doctor_address_visits x
#                      LEFT JOIN users y ON x.salesperson_fkey = y.id
#                      LEFT JOIN doctor_addresses z ON x.address_fkey = z.id
#                      LEFT JOIN doctors a ON z.doctor_fkey = a.id""", postg_conn)

ndf = xdf[['date', 'docid', 'doctor_name', 'salesperson', 'ReportingManager', 'notes']]
ndf.head()

,date,docid,doctor_name,salesperson,ReportingManager,notes
0,2025-01-16,CEN-01,HO,ARINDAM SAHA,NIREN KAUL,HO staff
1,2025-01-16,DOC-00924,SUBHASISH BISWAS,DIPANKAR ROY KARMAKAR,SUMAN PAUL,
2,2025-01-16,DOC-00924,SUBHASISH BISWAS,DIPANKAR ROY KARMAKAR,SUMAN PAUL,
3,2025-01-17,DOC-29443,SHASHANKA DHANUKA,SAYANTAN BOSE,SUMAN PAUL,
4,2025-01-17,DOC-13862,D S CHAKRABORTY,SAYANTAN BOSE,SUMAN PAUL,


# **Distance Covered / Day**

In [26]:
pip install gspread google-auth

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import gspread
from google.oauth2.service_account import Credentials

# Define the required scopes
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]

# Load credentials from the service account JSON file
creds = Credentials.from_service_account_file(
    r"C:\Users\suradmin\OneDrive - Suraksha Diagnostic Limited\Desktop\proj\tensile-jet-465507-v3-2d6ae440142a.json",
    scopes=scope
)

# Authorize the client
client = gspread.authorize(creds)

# Open the spreadsheet by its ID (not title!)
spreadsheet_id = "1QbDcA0jrBBKNdG-YQWRMLvrx_nqtnA54HkNEbhl2Hxw"
sheet = client.open_by_key(spreadsheet_id).sheet1

# Read all data as a list of dictionaries
data = sheet.get_all_records()

# Display the data

gsf = pd.DataFrame(data)
gsf.head()

,Sl No.,Sales Person,EMP ID,LAT,LONG,HOME ADDRESS,
0,1,PULAK MAITY,SUR04000,22.448474,88.429818,"unnamed road, Noapara, Rajpur Sonarpur - 70015...",
1,2,TANMAY SARKAR,SUR03698,22.684750,88.451474,"158/3 Bhattacharya Para, New Barrackpur. Kol- ...",
2,3,SAYANTAN BOSE,SUR03799,22.621302,88.421249,"Sumi Apartment, 194 Gorkhabasi Road, Nagerbaza...",
3,4,DIPANKAR ROY KARMAKAR,SUR02667,22.487886,88.365908,10/94/2B Bijoygarh kol-700092,
4,5,SOURADEEP BISWAS,SUR03408,22.737119,88.374095,"Near Shitala Mandir, Titaghar, barrackpore wes...",


In [28]:
zdf.head()

,date,salesperson_fkey,created_at,salesperson,ReportingManager,cust_type,doctor_fkey,docid,doctor_name,address,city,zipcode,longitude,latitude,distance_meters,location_status,notes,realtime,time_bucket
0,2025-01-16,237,2025-01-16 12:22:31.606730+00:00,ARINDAM SAHA,NIREN KAUL,DC,2865,CEN-01,HO,"DG block,New town,HO",Kolkata,700156,88.476172,22.579553,NaN,False,HO staff,17:52:31.606730,9:30AM-1:00PM
1,2025-01-16,222,2025-01-16 16:07:33.324867+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,SUBHASISH BISWAS,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",Kolkata,700092,88.363652,22.486545,NaN,False,,21:37:33.324867,1:00PM-5:00PM
2,2025-01-16,222,2025-01-16 16:11:26.025069+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,SUBHASISH BISWAS,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",Kolkata,700092,88.363638,22.486586,NaN,False,,21:41:26.025069,1:00PM-5:00PM
3,2025-01-17,223,2025-01-17 04:14:48.712002+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2251,DOC-29443,SHASHANKA DHANUKA,Ballygunge,Kolkata,700016,88.375949,22.526766,NaN,False,,09:44:48.712002,Beyond
4,2025-01-17,223,2025-01-17 05:21:31.511517+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2049,DOC-13862,D S CHAKRABORTY,36 ramkrishna lane,Kolkata,700078,88.378202,22.504825,NaN,False,,10:51:31.511517,Beyond


In [29]:
user = pd.read_sql("""SELECT id, username FROM users""", postg_conn)
user.head()

C:\Users\suradmin\AppData\Local\Temp\ipykernel_13728\3701281671.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user = pd.read_sql("""SELECT id, username FROM users""", postg_conn)


,id,username
0,38,SUR00143
1,9,admin1
2,31,SUR01351
3,33,SUR00465
4,35,SUR00686


In [30]:
mdf = pd.merge(zdf, user, left_on= 'salesperson_fkey', right_on= 'id', how= 'left')
mdf = mdf[['date', 'salesperson_fkey', 'id', 'username', 'created_at', 'salesperson', 'ReportingManager', 'cust_type', 'doctor_fkey', 'docid', 'doctor_name', 'address', 'city', 'zipcode', 'longitude',
           'latitude', 'distance_meters', 'location_status', 'notes', 'realtime']]
mdf.head()

,date,salesperson_fkey,id,username,created_at,salesperson,ReportingManager,cust_type,doctor_fkey,docid,doctor_name,address,city,zipcode,longitude,latitude,distance_meters,location_status,notes,realtime
0,2025-01-16,237,237,SUR03903,2025-01-16 12:22:31.606730+00:00,ARINDAM SAHA,NIREN KAUL,DC,2865,CEN-01,HO,"DG block,New town,HO",Kolkata,700156,88.476172,22.579553,NaN,False,HO staff,17:52:31.606730
1,2025-01-16,222,222,SUR02667,2025-01-16 16:07:33.324867+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,SUBHASISH BISWAS,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",Kolkata,700092,88.363652,22.486545,NaN,False,,21:37:33.324867
2,2025-01-16,222,222,SUR02667,2025-01-16 16:11:26.025069+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,SUBHASISH BISWAS,"97A, Southern Ave, Lake Market, Kalighat, Kolk...",Kolkata,700092,88.363638,22.486586,NaN,False,,21:41:26.025069
3,2025-01-17,223,223,SUR03799,2025-01-17 04:14:48.712002+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2251,DOC-29443,SHASHANKA DHANUKA,Ballygunge,Kolkata,700016,88.375949,22.526766,NaN,False,,09:44:48.712002
4,2025-01-17,223,223,SUR03799,2025-01-17 05:21:31.511517+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2049,DOC-13862,D S CHAKRABORTY,36 ramkrishna lane,Kolkata,700078,88.378202,22.504825,NaN,False,,10:51:31.511517


In [31]:
xyz = pd.merge(mdf, gsf, left_on= 'username', right_on= 'EMP ID', how= 'left')
xzx = xyz[['date', 'salesperson_fkey', 'id', 'username', 'created_at', 'salesperson', 'ReportingManager', 'cust_type', 'doctor_fkey', 'docid', 'doctor_name', 'address', 'city', 'zipcode', 'longitude',
           'latitude', 'distance_meters', 'location_status', 'notes', 'realtime', 'LAT', 'LONG']]
xzx.head()

,date,salesperson_fkey,id,username,created_at,salesperson,ReportingManager,cust_type,doctor_fkey,docid,...,city,zipcode,longitude,latitude,distance_meters,location_status,notes,realtime,LAT,LONG
0,2025-01-16,237,237,SUR03903,2025-01-16 12:22:31.606730+00:00,ARINDAM SAHA,NIREN KAUL,DC,2865,CEN-01,...,Kolkata,700156,88.476172,22.579553,NaN,False,HO staff,17:52:31.606730,NaN,NaN
1,2025-01-16,222,222,SUR02667,2025-01-16 16:07:33.324867+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,...,Kolkata,700092,88.363652,22.486545,NaN,False,,21:37:33.324867,22.487886,88.365908
2,2025-01-16,222,222,SUR02667,2025-01-16 16:11:26.025069+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,...,Kolkata,700092,88.363638,22.486586,NaN,False,,21:41:26.025069,22.487886,88.365908
3,2025-01-17,223,223,SUR03799,2025-01-17 04:14:48.712002+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2251,DOC-29443,...,Kolkata,700016,88.375949,22.526766,NaN,False,,09:44:48.712002,22.621302,88.421249
4,2025-01-17,223,223,SUR03799,2025-01-17 05:21:31.511517+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2049,DOC-13862,...,Kolkata,700078,88.378202,22.504825,NaN,False,,10:51:31.511517,22.621302,88.421249


In [32]:
# Sort by time to ensure the first visit comes first
df_sorted = xzx.sort_values(by=['username', 'created_at', 'realtime'])

# Drop duplicates to get first visit of each Saleid per date
first_visits = df_sorted.drop_duplicates(subset=['username', 'created_at'], keep='first')

# Create a new column for first visit latitude
# If you want to merge it back to the original df
fff = xzx.merge(first_visits[['username', 'created_at', 'latitude']],
                on=['username', 'created_at'],
                how='left',
                suffixes=('', '_FirstVisit'))

fff.head()

,date,salesperson_fkey,id,username,created_at,salesperson,ReportingManager,cust_type,doctor_fkey,docid,...,zipcode,longitude,latitude,distance_meters,location_status,notes,realtime,LAT,LONG,latitude_FirstVisit
0,2025-01-16,237,237,SUR03903,2025-01-16 12:22:31.606730+00:00,ARINDAM SAHA,NIREN KAUL,DC,2865,CEN-01,...,700156,88.476172,22.579553,NaN,False,HO staff,17:52:31.606730,NaN,NaN,22.579553
1,2025-01-16,222,222,SUR02667,2025-01-16 16:07:33.324867+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,...,700092,88.363652,22.486545,NaN,False,,21:37:33.324867,22.487886,88.365908,22.486545
2,2025-01-16,222,222,SUR02667,2025-01-16 16:11:26.025069+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,...,700092,88.363638,22.486586,NaN,False,,21:41:26.025069,22.487886,88.365908,22.486586
3,2025-01-17,223,223,SUR03799,2025-01-17 04:14:48.712002+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2251,DOC-29443,...,700016,88.375949,22.526766,NaN,False,,09:44:48.712002,22.621302,88.421249,22.526766
4,2025-01-17,223,223,SUR03799,2025-01-17 05:21:31.511517+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2049,DOC-13862,...,700078,88.378202,22.504825,NaN,False,,10:51:31.511517,22.621302,88.421249,22.504825


In [33]:
# Sort by time to ensure the first visit comes first
sorted_df = fff.sort_values(by= ['username', 'created_at', 'realtime'])

# Drop duplicates to get first visit of each Saleid per date
first_visit = sorted_df.drop_duplicates(subset= ['username', 'created_at'], keep= 'first')

# Create a new column for first visit latitude
# If you want to merge it back to the original df
fvf = fff.merge(first_visits[['username', 'created_at', 'longitude']],
                on=['username', 'created_at'],
                how='left',
                suffixes=('', '_FirstVisit')).rename(columns= {'LAT' : 'HomeLAT', 'LONG' : 'HomeLONG'})
fvf.head()

,date,salesperson_fkey,id,username,created_at,salesperson,ReportingManager,cust_type,doctor_fkey,docid,...,longitude,latitude,distance_meters,location_status,notes,realtime,HomeLAT,HomeLONG,latitude_FirstVisit,longitude_FirstVisit
0,2025-01-16,237,237,SUR03903,2025-01-16 12:22:31.606730+00:00,ARINDAM SAHA,NIREN KAUL,DC,2865,CEN-01,...,88.476172,22.579553,NaN,False,HO staff,17:52:31.606730,NaN,NaN,22.579553,88.476172
1,2025-01-16,222,222,SUR02667,2025-01-16 16:07:33.324867+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,...,88.363652,22.486545,NaN,False,,21:37:33.324867,22.487886,88.365908,22.486545,88.363652
2,2025-01-16,222,222,SUR02667,2025-01-16 16:11:26.025069+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,...,88.363638,22.486586,NaN,False,,21:41:26.025069,22.487886,88.365908,22.486586,88.363638
3,2025-01-17,223,223,SUR03799,2025-01-17 04:14:48.712002+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2251,DOC-29443,...,88.375949,22.526766,NaN,False,,09:44:48.712002,22.621302,88.421249,22.526766,88.375949
4,2025-01-17,223,223,SUR03799,2025-01-17 05:21:31.511517+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2049,DOC-13862,...,88.378202,22.504825,NaN,False,,10:51:31.511517,22.621302,88.421249,22.504825,88.378202


In [34]:
import numpy as np

# Earth's radius in kilometers
R = 6371.0

# Convert degrees to radians
lat1 = np.radians(fvf['HomeLAT'])
lon1 = np.radians(fvf['HomeLONG'])
lat2 = np.radians(fvf['latitude_FirstVisit'])
lon2 = np.radians(fvf['longitude_FirstVisit'])

# Differences
dlat = lat2 - lat1
dlon = lon2 - lon1

# Haversine formula
a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

# Final distance
fvf['Distance_Home_To_First'] = R * c
fvf.head()

,date,salesperson_fkey,id,username,created_at,salesperson,ReportingManager,cust_type,doctor_fkey,docid,...,latitude,distance_meters,location_status,notes,realtime,HomeLAT,HomeLONG,latitude_FirstVisit,longitude_FirstVisit,Distance_Home_To_First
0,2025-01-16,237,237,SUR03903,2025-01-16 12:22:31.606730+00:00,ARINDAM SAHA,NIREN KAUL,DC,2865,CEN-01,...,22.579553,NaN,False,HO staff,17:52:31.606730,NaN,NaN,22.579553,88.476172,NaN
1,2025-01-16,222,222,SUR02667,2025-01-16 16:07:33.324867+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,...,22.486545,NaN,False,,21:37:33.324867,22.487886,88.365908,22.486545,88.363652,0.275608
2,2025-01-16,222,222,SUR02667,2025-01-16 16:11:26.025069+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,...,22.486586,NaN,False,,21:41:26.025069,22.487886,88.365908,22.486586,88.363638,0.274371
3,2025-01-17,223,223,SUR03799,2025-01-17 04:14:48.712002+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2251,DOC-29443,...,22.526766,NaN,False,,09:44:48.712002,22.621302,88.421249,22.526766,88.375949,11.494959
4,2025-01-17,223,223,SUR03799,2025-01-17 05:21:31.511517+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2049,DOC-13862,...,22.504825,NaN,False,,10:51:31.511517,22.621302,88.421249,22.504825,88.378202,13.685152


In [35]:
# Sort by username, Date, and Time
sortdf = fvf.sort_values(by= ['username', 'date', 'realtime'])

# Initialize new column for previous latitude
fvf['Prev_Lat'] = np.nan

# Group by username and Date to track each salesperson's visits per day
grouped = sortdf.groupby(['username', 'date'])

# Function to find previous latitudes
def get_prev_latitudes(group):
    latitudes = []
    times = group['realtime'].tolist()
    lats = group['latitude'].tolist()

    for i in range(len(group)):
        if i == 0:
            latitudes.append(np.nan)  # No previous visit
        else:
            latitudes.append(lats[i - 1])
    return pd.Series(latitudes, index=group.index)

# Apply the function to each group
fvf.loc[df_sorted.index, 'Prev_Lat'] = grouped.apply(get_prev_latitudes).reset_index(level=[0,1], drop=True)
fvf.head()

C:\Users\suradmin\AppData\Local\Temp\ipykernel_13728\1698481544.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fvf.loc[df_sorted.index, 'Prev_Lat'] = grouped.apply(get_prev_latitudes).reset_index(level=[0,1], drop=True)


,date,salesperson_fkey,id,username,created_at,salesperson,ReportingManager,cust_type,doctor_fkey,docid,...,distance_meters,location_status,notes,realtime,HomeLAT,HomeLONG,latitude_FirstVisit,longitude_FirstVisit,Distance_Home_To_First,Prev_Lat
0,2025-01-16,237,237,SUR03903,2025-01-16 12:22:31.606730+00:00,ARINDAM SAHA,NIREN KAUL,DC,2865,CEN-01,...,NaN,False,HO staff,17:52:31.606730,NaN,NaN,22.579553,88.476172,NaN,NaN
1,2025-01-16,222,222,SUR02667,2025-01-16 16:07:33.324867+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,...,NaN,False,,21:37:33.324867,22.487886,88.365908,22.486545,88.363652,0.275608,NaN
2,2025-01-16,222,222,SUR02667,2025-01-16 16:11:26.025069+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,...,NaN,False,,21:41:26.025069,22.487886,88.365908,22.486586,88.363638,0.274371,22.486545
3,2025-01-17,223,223,SUR03799,2025-01-17 04:14:48.712002+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2251,DOC-29443,...,NaN,False,,09:44:48.712002,22.621302,88.421249,22.526766,88.375949,11.494959,NaN
4,2025-01-17,223,223,SUR03799,2025-01-17 05:21:31.511517+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2049,DOC-13862,...,NaN,False,,10:51:31.511517,22.621302,88.421249,22.504825,88.378202,13.685152,22.526766


In [36]:
# Sort by username, Date, and Time
dfsort = fvf.sort_values(by= ['username', 'date', 'realtime'])

# Initialize new column for previous longitude
fvf['Prev_Long'] = np.nan

# Group by username and Date to track each salesperson's visits per day
groupedx = dfsort.groupby(['username', 'date'])

# Function to find previous longitude
def get_prev_longitude(group):
    longitude = []
    times = group['realtime'].tolist()
    long = group['longitude'].tolist()

    for i in range(len(group)):
        if i == 0:
            longitude.append(np.nan)  # No previous visit
        else:
            longitude.append(long[i - 1])
    return pd.Series(longitude, index=group.index)

# Apply the function to each group
fvf.loc[dfsort.index, 'Prev_Long'] = grouped.apply(get_prev_longitude).reset_index(level=[0,1], drop=True)
fvf.head()

C:\Users\suradmin\AppData\Local\Temp\ipykernel_13728\1133000024.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fvf.loc[dfsort.index, 'Prev_Long'] = grouped.apply(get_prev_longitude).reset_index(level=[0,1], drop=True)


,date,salesperson_fkey,id,username,created_at,salesperson,ReportingManager,cust_type,doctor_fkey,docid,...,location_status,notes,realtime,HomeLAT,HomeLONG,latitude_FirstVisit,longitude_FirstVisit,Distance_Home_To_First,Prev_Lat,Prev_Long
0,2025-01-16,237,237,SUR03903,2025-01-16 12:22:31.606730+00:00,ARINDAM SAHA,NIREN KAUL,DC,2865,CEN-01,...,False,HO staff,17:52:31.606730,NaN,NaN,22.579553,88.476172,NaN,NaN,NaN
1,2025-01-16,222,222,SUR02667,2025-01-16 16:07:33.324867+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,...,False,,21:37:33.324867,22.487886,88.365908,22.486545,88.363652,0.275608,NaN,NaN
2,2025-01-16,222,222,SUR02667,2025-01-16 16:11:26.025069+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,...,False,,21:41:26.025069,22.487886,88.365908,22.486586,88.363638,0.274371,22.486545,88.363652
3,2025-01-17,223,223,SUR03799,2025-01-17 04:14:48.712002+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2251,DOC-29443,...,False,,09:44:48.712002,22.621302,88.421249,22.526766,88.375949,11.494959,NaN,NaN
4,2025-01-17,223,223,SUR03799,2025-01-17 05:21:31.511517+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2049,DOC-13862,...,False,,10:51:31.511517,22.621302,88.421249,22.504825,88.378202,13.685152,22.526766,88.375949


In [63]:
def distance_visit_to_visit(df: pd.DataFrame) -> pd.Series:
    
    R = 6371  # Earth's radius in KM

    # Convert degrees to radians, handling potential NaN values
    # np.radians handles NaN gracefully by returning NaN for those positions.
    lat1 = np.radians(df['Prev_Lat'])
    lon1 = np.radians(df['Prev_Long'])
    lat2 = np.radians(df['latitude'])
    lon2 = np.radians(df['longitude'])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Haversine formula components
    a = np.sin(dlat / 2) * np.sin(dlat / 2) + \
        np.cos(lat1) * np.cos(lat2) * \
        np.sin(dlon / 2) * np.sin(dlon / 2)

    
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance



# Calculate the distance
fvf['Distance_Visit_To_Visit'] = distance_visit_to_visit(fvf)
fvf.head()

,date,salesperson_fkey,id,username,created_at,salesperson,ReportingManager,cust_type,doctor_fkey,docid,...,notes,realtime,HomeLAT,HomeLONG,latitude_FirstVisit,longitude_FirstVisit,Distance_Home_To_First,Prev_Lat,Prev_Long,Distance_Visit_To_Visit
0,2025-01-16,237,237,SUR03903,2025-01-16 12:22:31.606730+00:00,ARINDAM SAHA,NIREN KAUL,DC,2865,CEN-01,...,HO staff,17:52:31.606730,NaN,NaN,22.579553,88.476172,NaN,NaN,NaN,NaN
1,2025-01-16,222,222,SUR02667,2025-01-16 16:07:33.324867+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,...,,21:37:33.324867,22.487886,88.365908,22.486545,88.363652,0.275608,NaN,NaN,NaN
2,2025-01-16,222,222,SUR02667,2025-01-16 16:11:26.025069+00:00,DIPANKAR ROY KARMAKAR,SUMAN PAUL,Doctor,1143,DOC-00924,...,,21:41:26.025069,22.487886,88.365908,22.486586,88.363638,0.274371,22.486545,88.363652,0.00485
3,2025-01-17,223,223,SUR03799,2025-01-17 04:14:48.712002+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2251,DOC-29443,...,,09:44:48.712002,22.621302,88.421249,22.526766,88.375949,11.494959,NaN,NaN,NaN
4,2025-01-17,223,223,SUR03799,2025-01-17 05:21:31.511517+00:00,SAYANTAN BOSE,SUMAN PAUL,Doctor,2049,DOC-13862,...,,10:51:31.511517,22.621302,88.421249,22.504825,88.378202,13.685152,22.526766,88.375949,2.45068


In [74]:
import pandas as pd
import numpy as np

from math import radians, sin, cos, sqrt, atan2

# Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in KM
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Ensure datetime columns
# df['created_at'] = pd.to_datetime(df['created_at'])
# df['TimeOnly'] = pd.to_datetime(df['TimeOnly'], format='%H:%M:%S').dt.time
# df['DateOnly'] = df['created_at'].dt.date

# Sort for sequence
dfxorted = fvf.sort_values(by=['username', 'date', 'realtime'])

# Group by salesperson + date
summary = []

for (salesperson, date), group in dfxorted.groupby(['salesperson', 'date']):
    group = group.sort_values(by='realtime').reset_index(drop=True)

    # Home coordinates
    home_lat = group['HomeLAT'].iloc[0]
    home_lon = group['HomeLONG'].iloc[0]
    username = group['username'].iloc[0]
    
    # First visit
    first_lat = group['latitude'].iloc[0]
    first_lon = group['longitude'].iloc[0]
    dist_home_to_first = haversine(home_lat, home_lon, first_lat, first_lon)

    # Last visit
    last_lat = group['latitude'].iloc[-1]
    last_lon = group['longitude'].iloc[-1]
    dist_last_to_home = haversine(last_lat, last_lon, home_lat, home_lon)

    # Visit-to-visit distances
    # visit_distance = 0
    # for i in range(1, len(group)):
    #     lat1, lon1 = group.loc[i - 1, ['latitude', 'longitude']]
    #     lat2, lon2 = group.loc[i, ['latitude', 'longitude']]
    #     visit_distance += haversine(lat1, lon1, lat2, lon2)

    daily_distance = (group.groupby(['salesperson', 'date'])['Distance_Visit_To_Visit']
    .sum()
    .reset_index(name='DailyDistance'))

    visit_distance = daily_distance['DailyDistance'].sum()

    total_distance = dist_home_to_first + visit_distance + dist_last_to_home

    summary.append({
        'Date': date,
        'Sales Person': salesperson,
        'username' : username,
        'Home_To_First_Visit(KM)': round(dist_home_to_first, 2),
        'Last_Visit_To_Home(KM)': round(dist_last_to_home, 2),
        'Total_Visit_Distance(KM)': round(visit_distance, 2),
        'Total_Distance_Covered(KM)': round(total_distance, 2)
    })

# Final summary DataFrame
summary_df = pd.DataFrame(summary)

# Optional: sort it
summary_df = summary_df.sort_values(by=['Date', 'Sales Person'])

summary_df

,Date,Sales Person,username,Home_To_First_Visit(KM),Last_Visit_To_Home(KM),Total_Visit_Distance(KM),Total_Distance_Covered(KM)
141,2025-01-16,ARINDAM SAHA,SUR03903,NaN,NaN,0.00,NaN
1173,2025-01-16,DIPANKAR ROY KARMAKAR,SUR02667,0.28,0.27,0.00,0.55
142,2025-01-17,AVIJIT SINGH,SUR03708,4.49,10.17,26.94,41.59
807,2025-01-17,DEBAJIT PAL CHOUDHURY,SUR03835,1.66,0.45,1.21,3.31
1174,2025-01-17,DIPANKAR ROY KARMAKAR,SUR02667,0.89,6.79,10.94,18.61
...,...,...,...,...,...,...,...
3277,2025-07-11,SOUVIK DEY,SUR00967,1.80,1.51,2.02,5.33
3396,2025-07-11,SUDIP CHAKRABORTY,SUR03665,5.03,5.38,18.13,28.54
3662,2025-07-11,SWARUP SAHA,SUR03162,1.71,1.92,8.19,11.82
3803,2025-07-11,TANMAY SARKAR,SUR03698,0.83,0.87,2.28,3.98


In [76]:
summary_df.to_csv(r"C:\Users\suradmin\OneDrive - Suraksha Diagnostic Limited\Desktop\DCovered.csv")